In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn import preprocessing
#from scipy.stats import shapiro
from scipy.cluster.hierarchy import dendrogram, linkage 
from scipy.cluster.hierarchy import fcluster
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
from google.cloud import bigquery
from google.oauth2 import service_account



In [3]:
start_date='2019-01-1'
end_date='2019-12-31'

In [4]:
cred_path='../../../../credential/Xpring Dev Sandbox-edb2a98acf17.json'

In [5]:
def get_gcp_creds(key_path):
    """
    Return GCP credentials from local .json
    
    Parameters
    ----------
    key_path: str
        Path to local credential .json
    """
    try:
        credentials = service_account.Credentials.from_service_account_file(
            key_path,
            scopes = ["https://www.googleapis.com/auth/cloud-platform"]
        )
    except:
        credentials = None
    return credentials;

In [6]:
def gbq_query(query,cred, query_params=None):
    """
    Run a query against Google Big Query, returning a pandas dataframe of the result.

    Parameters
    ----------
    query: str
        The query string
    cred: obj
        Credential class instance
    
    query_params: list, optional
        The query parameters to pass into the query string
    """
    client = bigquery.Client(credentials=cred)
    job_config = bigquery.QueryJobConfig()
    job_config.query_parameters = query_params
    return client.query(query, job_config=job_config).to_dataframe()

In [7]:
query1 = """
Select 
  TIMESTAMP(l.CloseTime) as `TimeStamp`,
  t.Account,t.Destination,t.Fee,t.TxnSignature,t.AmountXRP,t.LedgerIndex,t.TransactionType
FROM
  `xrpledgerdata.fullhistory.transactions`t
JOIN
  `xrpledgerdata.fullhistory.ledgers` l
  on t.LedgerIndex=l.LedgerIndex
where t.TransactionResult = "tesSUCCESS"
AND t.TransactionType ='AccountSet'

AND TIMESTAMP(l.CloseTime) >=TIMESTAMP(@start_date)
AND TIMESTAMP(l.CloseTime) <=TIMESTAMP(@end_date)


ORDER BY TimeStamp

LIMIT 100000;

"""
query_params1 = [
    bigquery.ScalarQueryParameter("start_date", "STRING", start_date),
    bigquery.ScalarQueryParameter("end_date", "STRING", end_date)
]



In [8]:
xrp1=gbq_query(query1,get_gcp_creds(cred_path),query_params1)

In [9]:
xrp1

,TimeStamp,Account,Destination,Fee,TxnSignature,AmountXRP,LedgerIndex,TransactionType
0,2019-01-01 07:32:31+00:00,r4AZpDKVoBxVcYUJCWMcqZzyWsHTteC4ZE,None,12,30450221008A0178B15CECC7BE391AA3C2A62A7AE102E8...,None,44100016,AccountSet
1,2019-01-01 08:27:30+00:00,r3daZLKpbk7mqwBVepDDjGW8sSnAVFa9YB,None,7500,304402207404DD21D368434F6AC0D4000D5BCB2AED5638...,None,44100875,AccountSet
2,2019-01-01 08:27:30+00:00,r3daZLKpbk7mqwBVepDDjGW8sSnAVFa9YB,None,7500,3045022100BB81C178DAD30EAE3DAA94DA6726C7ACD099...,None,44100875,AccountSet
3,2019-01-01 08:28:22+00:00,rPCNrPjfXUBY5VW8ikbbSUkiYPSGyYSpLM,None,7500,3044022031DF6D6FE5883D3F1A718DC4269DF7A97F9716...,None,44100890,AccountSet
4,2019-01-01 08:28:22+00:00,r3tUPKyx6YDx6TU3vXgAezz758rwyzfs8B,None,7500,3044022041D2EC6D50B8146957E2EBFC858701270BE758...,None,44100890,AccountSet
...,...,...,...,...,...,...,...,...
99995,2019-11-28 20:04:50+00:00,rEzaLebaTBXNy7C4s8Ru7yxxkE65bDAAkF,None,12,3043021F61C57826077D4198A193A3FC65C01D6AF97F97...,None,51705454,AccountSet
99996,2019-11-28 20:04:50+00:00,rEzaLebaTBXNy7C4s8Ru7yxxkE65bDAAkF,None,12,304502210099EE433380441A93BC0EB236A7B8C9FE6793...,None,51705454,AccountSet
99997,2019-11-28 20:04:50+00:00,rEzaLebaTBXNy7C4s8Ru7yxxkE65bDAAkF,None,12,3045022100A1B68C93547226AE4B201ABFA308D1A1067A...,None,51705454,AccountSet
99998,2019-11-28 20:04:50+00:00,rEzaLebaTBXNy7C4s8Ru7yxxkE65bDAAkF,None,12,3045022100DE1558340217CAE51938226D326CC0FF8E1D...,None,51705454,AccountSet


In [11]:
xrp1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
TimeStamp          100000 non-null datetime64[ns, UTC]
Account            100000 non-null object
Destination        0 non-null object
Fee                100000 non-null int64
TxnSignature       99997 non-null object
AmountXRP          0 non-null object
LedgerIndex        100000 non-null int64
TransactionType    100000 non-null object
dtypes: datetime64[ns, UTC](1), int64(2), object(5)
memory usage: 6.1+ MB


In [12]:
def dict_accounthash(data):
    txn_count=data[['Account','Destination']].apply(pd.value_counts).fillna(0).apply(lambda row : row.Account + row.Destination, axis =1)
    df_txn_count=txn_count.to_frame()
    df_txn_count.rename(columns={0:"Txn Count"})
    
    #Creating a merged a numpy array of unique account and destination hashes
    UniqueAccount=np.unique(data['Account'])
    UniqueDestination=np.unique(data['Destination'])
    UniqueHash=np.unique(np.concatenate([UniqueDestination, UniqueAccount]))
    #Creating a dictionary with account hashes as keys and account balances initialized at 0 as the values
    #The value is an array of length 3 for summing xrp received, xrp sent, and sum of fees
    UniqueHashList=UniqueHash.tolist()
    listofzeros = [[0] *3 ]*len(UniqueHashList)
    dicAccBal = {k:v for k,v in zip(UniqueHashList,listofzeros)}
    #Not sure why I need the line below, but otherwise I cannot do reassignment below, will instead make ever index the same value
    dicAccBal={k:[0,0,0] for (k,v) in dicAccBal.items()}

    #iterate through each transaction
    for index, row in data.iterrows():
        #Checking if the transaction's timestamp is within time window specified by starttime and endtime
        #Adding account balances to respective dictionary balance
        dicAccBal[row['Destination']][0] +=row['AmountXRP']
        dicAccBal[row['Account']][1] += row['AmountXRP']
        dicAccBal[row['Account']][2]+=row['Fee']
        
    df_sums=pd.DataFrame.from_dict(dicAccBal, orient='index',
                      columns=['Sum of XRP Received', 'Sum of XRP Sent', 'Sum of Fees'])
    full_df=df_sums.merge(df_txn_count,left_index=True,right_index=True)
    full_df.index.name='Account Hash'
    return full_df.rename(columns={0:"Txn Count"})

    